#  <center> Problem Set 4 <center>
<center> Spring 2021 <center>
<center> 3.100/3.322, 10.402/10.602, 20.301/20.401 <center>
<center> Due:10 pm ET on Thursday, Apr 22, 2021 <center>

## Part 2 SMILES-VAE

First, Request a GPU by going to Edit/Notebook Settings/Hardward Accelerator

In [ ]:
# install RDKIT 

# This is a hack to install rdkit without needing to install anaconda which might take minutes 
# if you have anaconda installed, you can directly install rdkit from anaconda 
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.7/dist-packages/

x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

# Install tqdm progress bar 
!pip install tqdm 

In [ ]:
# Optional: mount your google drive to save model and files 

from google.colab import drive
drive.mount('/content/drive')
mydrive = '/content/drive/MyDrive'

In [ ]:
import numpy as np
import torch
import torch.utils.data
from torch import nn, optim
import torch.nn.functional as F
import pandas as pd
from rdkit import Chem
from sklearn.preprocessing import LabelEncoder
import random
from tqdm import tqdm

from rdkit import RDLogger   
RDLogger.DisableLog('rdApp.*')

In [ ]:
# Get data 
! wget https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps4/data/zinc_50k.csv
    
# Get pretrained VAE model 
! wget -O vae_checkpoint.pth https://github.com/wwang2/ML4MolEng/blob/master/psets/ps4/pretrained_checkpoints/vae-050-0.06.pth?raw=true

### 1.1 One-hot encode SMILES strings into padded numerical vec-tors

In [ ]:
from sklearn import preprocessing

# Character list for SMILES string
moses_charset = ['2', 'o', 'C', 'I', 'O', 'H', 'n', 'N', '=', '+', '#', '-', 'c',
                 'B', 'l', '7', 'r', 'S', 's', '4', '6', '[', '5', ']', 'F', '3', 
                 'P', '(', ')', '1', ' ']

# Define encoder 
enc = preprocessing.LabelEncoder().fit(moses_charset)

# Read data 
df = pd.read_csv("./zinc_50k.csv")


Encode SMILES string into padded categorial vectors 

In [ ]:
################ Code #################




Make train/validation/test datasets

In [ ]:
################ Code #################





### 1.2 Implement the reparametrization trick for VAE



In [ ]:
# Molecular VAE model 

class MolVAE(nn.Module):
    def __init__(self,  rnn_enc_hid_dim, enc_nconv,
                         encoder_hid, z_dim, 
                         rnn_dec_hid_dim, dec_nconv, smiles_len, nchar
                         ):
        '''
            SMILES VAE model 
            
                rnn_enc_hid_dim: hidden dimension for the GRU encoder 
                enc_nconv: number of recurrent layers for the GRU decoder
                encoder_hid: dimension of GUR encoder readout
                z_dim: number of latent variable 
                rnn_dec_hid_dim: hidden dimension for the GRU decoder 
                dec_nconv: number of recurrent layers for the GRU decoder
                smiles_len: total length of padded SMILES string 
                nchar: number of possible characters 
                
        '''
        
        super(MolVAE, self).__init__()
        
        self.smiles_len = smiles_len
        self.nchar = nchar
        # Embedding layer
        self.embed = nn.Embedding(self.nchar, rnn_enc_hid_dim)
        # Encoding GRU
        self.rnn_enc = nn.GRU(rnn_enc_hid_dim, rnn_enc_hid_dim, enc_nconv, batch_first=True)
        # MLP to transfrom hidden output from Encoding GRU
        self.mlp0 = nn.Linear(rnn_enc_hid_dim, encoder_hid)
        # Network to parametrize mu
        self.mu_network = nn.Linear(encoder_hid, z_dim)
        # Network to parametrize log variance
        self.logvar_network = nn.Linear(encoder_hid, z_dim)
        # Decoding GRU
        self.rnn_dec = nn.GRU(z_dim, rnn_dec_hid_dim, dec_nconv, batch_first=True)
        # Output SMILES characters
        self.readout = nn.Linear(rnn_dec_hid_dim, self.nchar)

    def encode(self, x):
        '''output mean and logVariance of the encoded SMILES'''
        output, hn = self.rnn_enc(x)
        h = F.relu(self.mlp0(hn[-1]))
        return self.mu_network(h), self.logvar_network(h)
    
    def get_std(self, logvar):
        '''transfrom log variance to standard deviation'''
        ################ Code #################
        
        return logvar
        ################ Code #################

    def reparametrize(self, mu, std):
        '''the reparametrization trick'''
        if self.training:
            ################ Code #################
            
            
            return z
           ################ Code ################# 
        else:
            return mu

    def decode(self, z):
        '''decoder to reconstruct latent variable back to SMILES'''
        z = z.view(z.size(0), 1, z.size(-1)).repeat(1, self.smiles_len, 1)
        out, h = self.rnn_dec(z)
        out_reshape = out.contiguous().view(-1, out.size(-1))
        
        y0 = self.readout(out_reshape)
        y = y0.contiguous().view(out.size(0), -1, y0.size(-1))
        return y

    def forward(self, x):
        x_embed = self.embed(x) # Get SMILES embedding 
        mu, logvar = self.encode(x_embed) # Encoding SMILES to latent representations 
        std = self.get_std(logvar) # tranfrom log variance to std.
        z = self.reparametrize(mu, std) # reparametrization trick 
        smiles_recon = self.decode(z)  # reconstruct SMILES string 
        return smiles_recon, mu, std

Sample a 1d Gaussian distribution with mean 0, std = 1

In [ ]:
################ Code #################

# sample = 

In [ ]:
# Compare your sampling with N(0, 1)

import matplotlib.pyplot as plt
from scipy.stats import norm

plt.hist(sample.detach().cpu().numpy(), density=True)

# Plot a 1d unit Gaussian 
x_axis = np.arange(-7, 7, 0.001)
plt.plot(x_axis, norm.pdf(x_axis,0,1)) # Mean = 0, SD = 1.
plt.show()


### 1.3 Implement the SMILES VAE loss function

In [ ]:
def loss_function(recon_x, x, mu, var):
    ################ Code #################
    L_recon = 0.0  # reconstruction loss 
    L_kl = 0.0 # KL loss
    return L_recon, L_kl

In [ ]:
# Helper functions 
def index2smiles(mol_index, enc):
    '''Transform Your An array of character index back to SMILES'''
    smiles_charlist = enc.inverse_transform(mol_index)
    smiles = ''.join(smiles_charlist).strip(" ")
    
    return smiles

def check_smiles_valid(smiles):
    '''Check if SMILES string is valid'''
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        valid = True 
    else:
        valid = False 
    return valid

def sample_recon(model, epoch, dataloader, enc, n_sample=500):
    '''Randomly sample from data loader to get reconstructed SMILES and test if it is valid'''
    # evaluation mode 
    model.eval()
    
    # Randomly select samples 
    random_mol = random.choices( range( len(dataloader.dataset) ), k=n_sample)
    x_mol = train_loader.dataset.__getitem__(random_mol)[0]
    x_mol = x_mol.to(device)
    
    # Get reconstructed Molecule 
    mol_recon, mu, logvar = model(x_mol)
    mol_index = mol_recon.detach().cpu().numpy().argmax(-1).squeeze()
    mol_orig = x_mol.detach().cpu().numpy()
    
    # Show the first reconstructed SMILES
    print("epoch {} sample orig SMILES: {}".format(epoch,  index2smiles(mol_orig[0], enc)))
    print("epoch {} sample recon SMILES: {}".format(epoch,  index2smiles(mol_index[0], enc)))
    
    valid_count = 0 
    for indexed_smiles in mol_index:
        smiles = index2smiles(indexed_smiles, enc)
        valid = check_smiles_valid(smiles)
        
        if valid: 
            valid_count += 1
    
    print("{}/{} SMILES are valid".format(valid_count, n_sample))

def loop(model, loader, epoch, beta=0.05, evaluation=False):
    '''
        Train/test your VAE model
    '''
    
    if evaluation:
        model.eval()
        mode = "eval"
    else:
        model.train()
        mode = 'train'
    batch_losses = []
        
    tqdm_data = tqdm(loader, position=0, leave=True, desc='{} (epoch #{})'.format(mode, epoch))
    for data in tqdm_data:
        
        x = data[0].to(device)
        recon_batch, mu, std = model(x)
        loss_recon, loss_kl = loss_function(recon_batch, x, mu, std)
        loss = loss_recon + beta * loss_kl     
        
        if not evaluation:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        batch_losses.append(loss.item())

        postfix = ['recon loss={:.3f}'.format(loss_recon.item()) ,
                   'KL loss={:.3f}'.format(loss_kl.item()) ,
                   'total loss={:.3f}'.format(loss.item()) , 
                   'avg. loss={:.3f}'.format(np.array(batch_losses).mean())]
        
        tqdm_data.set_postfix_str(' '.join(postfix))
    
    return np.array(batch_losses).mean()

### 1.4  Train your model

In [ ]:
device = 0

model = MolVAE(rnn_enc_hid_dim=367, enc_nconv=2, 
                     encoder_hid=512, z_dim=171, rnn_dec_hid_dim=512,
                    dec_nconv=1, nchar=31, smiles_len=max_len)
                      
model = model.to(device)

# load pretrained model 
model.load_state_dict(torch.load("./vae_checkpoint.pth"))

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=5e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

In [ ]:
epochs = 1000

for epoch in range(0, epochs):
    
    sample_recon(model, epoch, val_loader, enc)
    train_loss = loop(model, train_loader, epoch)
    val_loss = loop(model, val_loader, epoch, evaluation=True)
    scheduler.step(val_loss)
    
    # optional: save model 
#     if epoch % 15 == 0:
#         torch.save(model.state_dict(),
#                 './{}/vae-{:03d}-{:.2f}.pth'.format(mydrive, epoch, train_loss))
        
#         torch.save(optimizer.state_dict(),
#             './{}/optim-{:03d}-{:.2f}.pth'.format(mydrive, epoch, train_loss))

    if epoch == 0:
        best_loss = train_loss.item()
    else:
        if train_loss.item() < best_loss:
            best_loss = train_loss.item()


### 1.5 Sample new molecules 

In [ ]:
################ Code #################





Visualize molecular line drawings for the molecules you generated.

In [ ]:
################ Code #################